In [1]:
import os,threading
from pathlib import Path

from flask import Flask, request, abort
import logging
from logging.handlers import RotatingFileHandler

import IMdb_crawler
import kimo
import rottenTomatos

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, ImageMessage, TextSendMessage, ImageSendMessage,ImageCarouselColumn,MessageTemplateAction,
    TemplateSendMessage,ImageCarouselTemplate,URITemplateAction,ImagemapArea,QuickReplyButton,QuickReply,PostbackAction,
    MessageAction
)
import json

from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials

import pymysql

app = Flask(__name__)

#app.config['DEBUG'] = True

#與 mysql進行連線
conn = pymysql.Connect(host='localhost',
                        port=3306,
                        user='root',
                        passwd='',
                        db='TFB103d_azure',
                        charset='utf8')
cur = conn.cursor()

secretFile = json.load(open("secretFile.txt",'r'))
channelAccessToken = secretFile['channelAccessToken']
channelSecret = secretFile['channelSecret']

#與 Line Bot Api 連線
line_bot_api = LineBotApi(channelAccessToken)
handler = WebhookHandler(channelSecret)
ngrokUrl = 'https://17f9-111-235-228-63.ngrok.io/'


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

#接收文字資訊 (功能文字以 @ 開頭；電影名稱以 # 開頭)
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    if event.message.text == '@熱門電影':
        message = hotMovie(event)
        
    elif event.message.text == '@最新電影':
        message = newMovie(event)
        
    elif event.message.text == '@訂票連結':
        message = movieLink(event)
        
    elif event.message.text == '@重新查詢':
        userSessionDict[event.source.user_id]=set()
        message = TextSendMessage("已清除資訊，請重新查詢。")
              
    elif '#' in event.message.text:
        message = movieDesc(event.message.text)
    
    else:    
        message = TextSendMessage(
        text="未查詢到電影，請再提供圖片進行查詢。",
        quick_reply=QuickReply(
            items=[
                QuickReplyButton(
                    action=MessageAction(label="重新查詢",data="@重新查詢")
                ),
            ]))
    
    line_bot_api.reply_message(
        event.reply_token,message
        )

#接收圖片搜尋
@handler.add(MessageEvent, message=ImageMessage)
def handle_image_message(event):
    message_id = event.message.id
#     app.logger.info("message_id: " + message_id)

    # 從message_id取得image的內容
    message_content = line_bot_api.get_message_content(message_id)

    with open(Path(f"static/images/{message_id}.jpg").absolute(), "wb") as f: #,encoding="utf-8"
        # 存入本機位置 (用 path()..absolute()取得絕對路徑)
        for chunk in message_content.iter_content():
            f.write(chunk)
    
    #讀取照片進行人臉辨識，回傳名單為 list
    actorList = faceDetection(message_id)
    
    if(actorList == None): #代表正在爬蟲，爬蟲時間過長會超過 line bot 接收訊息的時間，故先回應訊息
        line_bot_api.reply_message(event.reply_token, TextSendMessage("電影資料抓取中，請稍候 5 分鐘再重新嘗試。"))
        
    if(len(actorList) == 0):
        line_bot_api.reply_message(event.reply_token, TextSendMessage("未偵測到演員，請重新上傳。"))
        
    elif(len(actorList) == 1):
        selectSql='''SELECT movie_name from movie_actor_ref 
                     WHERE actor_name = '%s' 
                     GROUP BY movie_name
                     LIMIT 5;''' % (actorList[0])
    else:
        selectSql='''SELECT movie_name 
                     FROM movie_actor_ref 
                     WHERE actor_name in %s
                     GROUP BY movie_name 
                     LIMIT 5;''' % (tuple(actorList),)
        
    print(selectSql)
    cur.execute(selectSql)
    result_set = cur.fetchall()
    movieList=[]    
    for row in result_set:
        movieList.append(str(row[0]))
        
    print(len(movieList))
    print(movieList)
    
    if(len(movieList) == 0):
        message = TextSendMessage(
                        text="未查詢到電影，請再提供圖片進行查詢，或點選重新查詢清空重查。",
                        quick_reply=QuickReply(
                            items=[
                                QuickReplyButton(
                                    action=MessageAction(label="重新查詢",text="@重新查詢")
                                ),
                            ]))
        line_bot_api.reply_message(event.reply_token, message)
            
    elif(len(movieList) == 1):
        messageList = movieDesc(movieList[0])
        line_bot_api.reply_message(event.reply_token, messageList)
            
    else:
    #產成圖片選單
        
        columnsList=[]
        for movie in movieList:
            selectMovieSql='''select movie_id, link, movie_name from movie where movie_name = %s;'''
            cur.execute(selectMovieSql,(movie))
            movie_results = cur.fetchall()
        
            for row in movie_results:
                uuid=row[0]
                link=row[1]
                labelStr=row[2]
                textStr = "#"+labelStr
                if(len(labelStr)>12):
                    labelStr=labelStr[:9]+"..."
                image_cc = ImageCarouselColumn(
                            image_url = ngrokUrl + f"static/images/{uuid}.jpg",
                            action = MessageTemplateAction(
                                label = labelStr
                                ,text = textStr))

                columnsList.append(image_cc)

        message = [TemplateSendMessage(
                alt_text='電影列表',
                template=ImageCarouselTemplate(
                    columns = columnsList
                )),
                TextSendMessage(
                    text = "查詢結果如上列表：\n可再上傳圖片查詢，或是點擊圖片查詢電影資訊。"
                )]
        
        line_bot_api.reply_message(event.reply_token, message)
                        
    
# 透過 Azure ComputerVision
def faceDetection(messageId):
    # Set API key.
    subscription_key = 'df0710b268d64a9aa41637d0dbf24891'
    # Set endpoint.
    endpoint = 'https://lin19911113pro.cognitiveservices.azure.com/'
    # Call API
    computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
    print("===== Detect Domain-specific Content - remote =====")
    
    # Call API with content type 名人識別 (celebrities) and URL
    detect_domain_results_celebs_remote = computervision_client.analyze_image_by_domain_in_stream(
        "celebrities", open(f"./static/images/{messageId}.jpg", "rb"))
    
    nameList = list()
    # Print detection results with name
    print("Celebrities in the remote image:")
    if len(detect_domain_results_celebs_remote.result["celebrities"]) == 0:
        print("No celebrities detected.")
    else:
        for celeb in detect_domain_results_celebs_remote.result["celebrities"]:
            app.logger.info(celeb["name"])
            nameList.append(celeb["name"])
            
            #確認資料庫中已經爬過，已爬過便不再爬
            selectSql='''select count(actor_name) from movie_actor_ref where actor_name = %s ;'''
            cur = conn.cursor()
            cur.execute(selectSql,celeb["name"])
            number_of_actor = cur.fetchone()[0]
            
            if(int(number_of_actor) == 0):
                def movie_thread():
                    try: #啟動爬蟲抓取該演員演出之電影資訊
                        IMdb_crawler.IMDb_crawler_by_python(celeb["name"],cur)
                        kimo.KiMoScore_Desc(celeb["name"],cur)
                        rottenTomatos.rotten_tomato_score(celeb["name"],cur)
                    finally:
                        conn.commit()
                # 建立一個子執行緒
                t = threading.Thread(target = movie_thread)

                # 執行該子執行緒
                t.start()
                
                return None
    return nameList

# 熱門電影排行(以被查詢次數為準)
def hotMovie(event):
    try:
        sql = '''SELECT movie_id,link,movie_name
                 FROM movie
                 ORDER BY click_count DESC
                 LIMIT 5;
               '''
        cur.execute(sql)
        result = cur.fetchall()
        ans = ''
        columnsList=[]
        for row in result:
            movie_id=row[0]
            link=row[1]
            labelStr=row[2]
            textStr = "#"+labelStr
            if(len(labelStr)>12):
                labelStr=labelStr[:9]+"..."
            image_cc = ImageCarouselColumn(
                        image_url=ngrokUrl+f"static/images/{movie_id}.jpg",
                        action=MessageTemplateAction(
                                label=labelStr,
                                text=textStr)
            )
            columnsList.append(image_cc)

        return [TemplateSendMessage(
                alt_text='熱門電影排行',
                template=ImageCarouselTemplate(
                    columns=columnsList
                )),
                TextSendMessage(
                    text = "查詢結果如上：\n可再上傳圖片查詢，或是點擊圖片查詢電影資訊。"
                )]
    except:
        return TextSendMessage(text='發生錯誤！')

# 最新上映電影排行
def newMovie(event):
    try:
        sql = '''SELECT movie_id,link,movie_name
                 FROM movie
                 WHERE release_date < '2022-01-01' 
                 ORDER BY release_date DESC
                 LIMIT 5;
               '''
        cur.execute(sql)
        result = cur.fetchall()
        ans = ''
        columnsList=[]
        for row in result:
            movie_id=row[0]
            link=row[1]
            labelStr=row[2]
            textStr = "#"+labelStr
            if(len(labelStr)>12):
                labelStr=labelStr[:9]+"..."
            image_cc=ImageCarouselColumn(
                        image_url=ngrokUrl+f"static/images/{movie_id}.jpg",
                        action=MessageTemplateAction(
                                label=labelStr,
                                text=textStr
                            )
                    )
            columnsList.append(image_cc)

        return [TemplateSendMessage(
                alt_text='最新電影',
                template=ImageCarouselTemplate(
                    columns=columnsList
                )),
                TextSendMessage(
                    text = "查詢結果如上：\n可再上傳圖片查詢，或是點擊圖片查詢電影資訊。"
                )]
    except:
        return TextSendMessage(text='發生錯誤！')

def movieLink(event):
    try:

        return [TemplateSendMessage(
                alt_text='訂票連結',
                template= ImageCarouselTemplate(
                    columns= [
                    ImageCarouselColumn(
                        image_url = ngrokUrl+"static/images/weishow.JPG",
                        baseSize = {"width":50, "height":50},
                        action = URITemplateAction(
                                label="華納威秀",
                                uri="https://www.vscinemas.com.tw/"
                            )),
                    ImageCarouselColumn(
                        image_url=ngrokUrl+"static/images/ez.JPG",
                        action=URITemplateAction(
                                label="ez訂",
                                uri="https://www.ezding.com.tw/"
                            )),
                    ImageCarouselColumn(
                        image_url=ngrokUrl+"static/images/amba.JPG",
                        action=URITemplateAction(
                            label="國賓影城",
                            uri="https://www.ambassador.com.tw/"
                            ))
                            ]
                        )
                    ),
                TextSendMessage(
                    text = "點擊圖片即可進入訂票網站。"
                )]
    except:
        return TextSendMessage(text='發生錯誤！')
        
        
def movieDesc(movie_name):
    
    movie_name = movie_name.replace('#', '')
    
    #查詢次數計次加一
    updateClickSQL=f"UPDATE movie SET click_count = click_count + 1 where movie_name = '{movie_name}';"
    cur.execute(updateClickSQL)
    print(updateClickSQL)
    conn.commit()
    
    
    try:
        sql = '''SELECT movie_id, link, movie_name, movie_desc, imdb_rating, tomato_rating, kimo_rating
                 FROM movie
                 where movie_name = '%s';
               '''%(movie_name)
        cur.execute(sql)
        result = cur.fetchall()
        columnsList=[]
        for row in result:
            #圖片訊息
            movie_id=row[0]
            link=row[1]
            labelStr=row[2]
            textStr = "#"+labelStr
            if(len(labelStr)>12):
                labelStr=labelStr[:9]+"..." #Label有顯示長度限制
            image_cc=ImageCarouselColumn(
                        image_url=ngrokUrl+f"static/images/{movie_id}.jpg",
                        action=URITemplateAction(
                                label=labelStr,
                                uri=link
                            )
                    )
            columnsList.append(image_cc)
            
            #文字訊息
            try:
                score = float(row[4]+row[5]+(row[6]*2)) / 3
            except:
                score = 0.0
            text_message = "您查詢的電影名稱為：\n%s \n平均分數：%.1f\n" % (movie_name,score) + "電影簡介為：%s" % row[3]
            
        return [TemplateSendMessage(
                alt_text='電影資訊',
                template=ImageCarouselTemplate(
                    columns=columnsList)),
                TextSendMessage(
                    text = text_message,
                    quick_reply=QuickReply(
                        items=[
                            QuickReplyButton(
                                action=MessageAction(label="重新查詢",text="@重新查詢"))
                        ]))]
    except:
        return [TextSendMessage(text='發生錯誤！', quick_reply = QuickReply(
                            items=[
                                QuickReplyButton(
                                    action=MessageAction(label="重新查詢",text="@重新查詢")
                                )]))]   
        
if __name__ == "__main__":
    userSessionDict = dict()
    formatter = logging.Formatter(
        "[%(asctime)s][%(filename)s:%(lineno)d][%(levelname)s][%(thread)d] - %(message)s")
    loghandler = RotatingFileHandler("flask.log", maxBytes=1024000, backupCount=10)
    app.logger.addHandler(loghandler)
    app.logger.setLevel(logging.INFO)
    loghandler.setFormatter(formatter)
    app.run(host='0.0.0.0',port='6868')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:6868/ (Press CTRL+C to quit)
[2021-11-16 14:22:59,638] INFO in <ipython-input-1-4374de074a24>: Request body: {"destination":"Uf57b3926faee6d8dd66ecca1d4a677b3","events":[{"type":"message","message":{"type":"image","id":"15092056560260","contentProvider":{"type":"line"}},"timestamp":1637043778187,"source":{"type":"user","userId":"U94d88cfd8e0c8243ef4e92f754fc62f6"},"replyToken":"23e365bc273e4af798454e738881a606","mode":"active"}]}


===== Detect Domain-specific Content - remote =====


[2021-11-16 14:23:02,019] INFO in <ipython-input-1-4374de074a24>: Dong-seok Ma
[2021-11-16 14:23:02,022] INFO in <ipython-input-1-4374de074a24>: Richard Madden
[2021-11-16 14:23:02,025] INFO in <ipython-input-1-4374de074a24>: Angelina Jolie
127.0.0.1 - - [16/Nov/2021 14:23:02] "POST / HTTP/1.1" 200 -


Celebrities in the remote image:
SELECT movie_name 
                     FROM movie_actor_ref 
                     WHERE actor_name in ('Dong-seok Ma', 'Richard Madden', 'Angelina Jolie')
                     GROUP BY movie_name 
                     LIMIT 5;
5
['Eternals', 'Those Who Wish Me Dead', 'Come Away', 'Kung Fu Panda 3', 'Maleficent']


...

[2021-11-16 16:14:08,345] INFO in <ipython-input-1-4374de074a24>: Timothée Chalamet
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Tibame\anaconda3\lib\logging\__init__.py", line 1084, in emit
    stream.write(msg + self.terminator)
UnicodeEncodeError: 'cp950' codec can't encode character '\xe9' in position 82: illegal multibyte sequence
Call stack:
  File "C:\Users\Tibame\anaconda3\lib\threading.py", line 890, in _bootstrap
    self._bootstrap_inner()
  File "C:\Users\Tibame\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\Tibame\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Tibame\anaconda3\lib\socketserver.py", line 650, in process_request_thread
    self.finish_request(request, client_address)
  File "C:\Users\Tibame\anaconda3\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\Users\

Celebrities in the remote image:
SELECT movie_name from movie_actor_ref 
                     WHERE actor_name = 'Timothée Chalamet' 
                     GROUP BY movie_name
                     LIMIT 5;
5
['Dune', 'The French Dispatch', 'Little Women', 'A Rainy Day in New York', 'Beautiful Boy']


127.0.0.1 - - [16/Nov/2021 16:14:09] "GET /static/images/63f53dd1-4691-11ec-9027-002b6760c0e9.jpg HTTP/1.1" 200 -
[2021-11-16 16:14:12,182] INFO in <ipython-input-1-4374de074a24>: Request body: {"destination":"Uf57b3926faee6d8dd66ecca1d4a677b3","events":[{"type":"message","message":{"type":"text","id":"15092554992000","text":"#Dune"},"timestamp":1637050451085,"source":{"type":"user","userId":"U94d88cfd8e0c8243ef4e92f754fc62f6"},"replyToken":"bca2c090d1ae4e59adfd76152f89cc4e","mode":"active"}]}
127.0.0.1 - - [16/Nov/2021 16:14:12] "POST / HTTP/1.1" 200 -


UPDATE movie SET click_count = click_count + 1 where movie_name = 'Dune';


[2021-11-16 16:14:15,829] INFO in <ipython-input-1-4374de074a24>: Request body: {"destination":"Uf57b3926faee6d8dd66ecca1d4a677b3","events":[{"type":"message","message":{"type":"text","id":"15092555290622","text":"@重新查詢"},"timestamp":1637050454952,"source":{"type":"user","userId":"U94d88cfd8e0c8243ef4e92f754fc62f6"},"replyToken":"8e0fe308237143808f18bfad4cc161f5","mode":"active"}]}
127.0.0.1 - - [16/Nov/2021 16:14:15] "POST / HTTP/1.1" 200 -
[2021-11-16 16:14:40,498] INFO in <ipython-input-1-4374de074a24>: Request body: {"destination":"Uf57b3926faee6d8dd66ecca1d4a677b3","events":[{"type":"message","message":{"type":"text","id":"15092557182312","text":"@熱門電影"},"timestamp":1637050479126,"source":{"type":"user","userId":"U94d88cfd8e0c8243ef4e92f754fc62f6"},"replyToken":"6f78f40114d148d8b4ac364adb6c3730","mode":"active"}]}
127.0.0.1 - - [16/Nov/2021 16:14:40] "POST / HTTP/1.1" 200 -
[2021-11-16 16:14:50,431] INFO in <ipython-input-1-4374de074a24>: Request body: {"destination":"Uf57b3926fae

In [1]:
!pip install --upgrade azure-cognitiveservices-vision-computervision
!pip install bs4
!pip install pymysql